![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx and google `flan-t5-xxl` to generate advertising

#### Disclaimers

- Foundational Model REST API that is used in this notebook is in `beta` state.
- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support of text generation in Watsonx. It introduces commands for data generation and model testing.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to use `flan-t5-xxl` model to generate mail advertising


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Data loading](#data)
- [Foundation Models on Watsonx](#models)
- [Model testing](#predict)
- [Summary](#summary)

<a id="setup"></a>
##  Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).


### Install and import the `datasets` and dependecies

In [ ]:
!pip install requests | tail -n 1
!pip install wget | tail -n 1
!pip install ibm-cloud-sdk-core | tail -n 1
!pip install wget | tail -n 1

In [18]:
import os, getpass, json
import requests
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator

### Inferencing class
This cell defines a class that makes a REST API call to the watsonx Foundation Model
inferencing API that we will use to generate output from the provided input.
The class takes the access token created in the previous step, and uses it to
make a REST API call with input, model id and model parameters. The response
from the API call is returned as the cell output.

**Action:** Provide Watson Machine Learning url to work with watsonx.ai.

In [3]:
endpoint_url = input("Please enter your WML endpoint url (hit enter): ")

Please enter your WML endpoint url (hit enter): ········


Define a `Prompt` class for prompts generation.

In [4]:
class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = f"{endpoint_url}/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]
        else:
            return response.text

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [5]:
access_token = IAMTokenManager(
    apikey = getpass.getpass("Please enter your WML api key (hit enter): "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

Please enter your WML api key (hit enter): ········


### Defining the project id
The API requires project id that provides the context for the call. We will obtain
the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [6]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

Please enter your project_id (hit enter): ········


<a id="data"></a>
## Advertising prompts

#### Prepare advertising prompts.

Prepare credit card with cashback advertising.

In [7]:
ads = []

ads.append("Generate banking advertising mail of a credit card with 10% cashback.")

Prepare savings account advertising.

In [8]:
ads.append("Generate banking advertising mail of a saving account with 8 percent interest.")

Prepare educational course advertising.

In [9]:
ads.append("Generate advertising mail of an online JAVA programming course.")

Prepare photo editing software advertising.

In [10]:
ads.append("Generate advertising mail of paid (10$ a month) photo editing software for professionals.")

Prepare healthy food catering advertising.

In [11]:
ads.append("Generate advertising mail of healthy food catering with a free first meal.")

<a id="models"></a>
## Foundation Models on Watsonx

#### List available models

In [12]:
models_json = requests.get(endpoint_url + '/ml/v1-beta/foundation_model_specs?version=2022-08-01&limit=50',
                           headers={
                                    'Authorization': f'Bearer {access_token}',
                                    'Content-Type': 'application/json',
                                    'Accept': 'application/json'
                            }).json()
models_ids = [m['model_id'] for m in models_json['resources']]
print(models_ids)

['bigscience/mt0-xxl', 'eleutherai/gpt-neox-20b', 'google/flan-t5-xxl', 'google/flan-ul2', 'ibm/mpt-7b-instruct2']


You need to specify `model_id` that will be used for inferencing:

In [13]:
model_id = "google/flan-t5-xxl"

<a id="predict"></a>
##  Generate mail advertising

### Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [14]:
parameters =  {"decoding_method": "sample",
               "temperature": 0.3,
               "top_p": 1,
               "top_k": 50,
               "random_seed": 33,
               "repetition_penalty": 2,
               "min_new_tokens": 50,
               "max_new_tokens": 1024
}

### Generate mail advertising using `flan-t5-xxl` model.


**Note:** You might need to adjust model `parameters` for different models or tasks, to do so please refer to [documentation]().

Initialize the `Prompt` class.

**Hint:** Your authentication token might expire, if so please regenerate the `access_token` and reinitialize the `Prompt` class.

In [15]:
prompt = Prompt(access_token, project_id)

Get the docs summaries.

In [16]:
results_low_temperature = []

for instruction in ads:
    results_low_temperature.append(prompt.generate(instruction, model_id, parameters))

Explore model outputs.

In [19]:
print(json.dumps(results_low_temperature, indent=2))

[
  {
    "generated_text": "The Bank of America Cash Rewards Credit Card offers a 10% cash back on all purchases. Earn points for every $1 you spend. Use your rewards to get travel, merchandise, and gift cards. Get rewarded for shopping online, at the gas pump, or anywhere else you shop.",
    "generated_token_count": 58,
    "input_token_count": 16,
    "stop_reason": "EOS_TOKEN",
    "seed": 33
  },
  {
    "generated_text": "a saving account with 8 percent interest. if you're interested, please call us at 1-800-447-2337 or visit any of our banking centers. id#123456. cc#5feb3d",
    "generated_token_count": 53,
    "input_token_count": 16,
    "stop_reason": "EOS_TOKEN",
    "seed": 33
  },
  {
    "generated_text": "Dear Sir/Madam, We are offering an online JAVA programming course. The course is designed by a team of experienced and qualified professionals. You can learn to program in Java and get certified in the process. Please visit our website for more information. Thank you,"

You might also try to change model parameters to see whether it provides better ads. In following cell we changed the model temperature to 1.0 to let the model select words more creatively.

In [20]:
parameters_temp =  {"decoding_method": "sample",
                    "temperature": 1.0,
                    "top_p": 1,
                    "top_k": 50,
                    "random_seed": 33,
                    "repetition_penalty": 2,
                    "min_new_tokens": 50,
                    "max_new_tokens": 1024
}

In [21]:
results_high_temperature = []

for instruction in ads:
    results_high_temperature.append(prompt.generate(instruction, model_id, parameters_temp))

Explore model outputs.

In [23]:
print(json.dumps(results_high_temperature, indent=2))

[
  {
    "generated_text": "From now on, enjoy some amazing advantages with Sainsbury's Premier 1 Cashback Card! Redeem your card at any one of our 200+ retailers and get 10% cash back! You need to apply for this incredible offer here. Don't delay",
    "generated_token_count": 53,
    "input_token_count": 16,
    "stop_reason": "EOS_TOKEN",
    "seed": 33
  },
  {
    "generated_text": "From the Credit Union Board of Directors: All members who have been with our credit union for a year or more are eligible for an 8.00% interest checking account. To open yours, stop by any of our locations, apply on-line at www.rbcusa.com or by calling 1-800-457-5555. For over 100 years we've made banking easy.",
    "generated_token_count": 81,
    "input_token_count": 16,
    "stop_reason": "EOS_TOKEN",
    "seed": 33
  },
  {
    "generated_text": "From the makers of JAVA: \"Avail yourself of our online course and learn the programming language from a pro-teacher and the best technical team on the 

You can also try changing the prompt and/or parameters to generate more detailed advertising. In the next cell, the photo editing software advertising prompt was transformed to provide more accurate results.

In [24]:
photo_software_prompt = "Write an advertising e-mail of paid (9$ a month, first month free) photo editing software COOLSOFTWARE targeted for early professionals."

In [25]:
parameters_photo =  {"decoding_method": "sample",
                    "temperature": 1.0,
                    "top_p": 1,
                    "top_k": 50,
                    "random_seed": 33,
                    "repetition_penalty": 2,
                    "min_new_tokens": 200,
                    "max_new_tokens": 1024
}

In [26]:
results_photo_soft = prompt.generate(photo_software_prompt, model_id, parameters_photo)

In [27]:
print(json.dumps(results_photo_soft, indent=2))

{
  "generated_text": "i discovered photography was my hobby - not profession. until now that is. with the introduction of new technology, there is no limit to what you can do with your photos. now, all you basic need for photo editing is this great software called pixlr xpress, which you may find at http://www.softeventsandmedia.com/?cat=265. i just wanted to let you know that there is an amazing deal for early photographers: you can get 1 month free. after that, the subscription goes up by 9 dollars every month. we are currently on sale, and if you are interested, you should order your copy right away. otherwise, you wont be in a position to try it out, because it is only available in demo version. a bit over 100 free samples are already available, though. you dont have to buy anything first, so grab your free trial right now. as soon as you try it out, youll be pleased to hear that the interface is extremely intuitive and its fairly easy to edit your photo with it.",
  "generated_to

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to generate mail advertising with Google's `flan-t5-xxl` on Watsonx. 
 
 Check out our _[Online Documentation]()_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Szymon Kucharczyk**, Staff Software Engineer at Watson Machine Learning.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.